In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv(r'C:\Users\Avery\OneDrive\Documents\GitHub\GGE6102\Data_Sept_F180_Combined.csv')

df

,Time (sec),latitude(deg),longitude(deg),X (m),Y (m),height(m),sdn(m),sde(m),sdu(m),Vn(km/h),Vn (m/s),Ve(km/h),Ve(m/s),Vd(km/h),Vu(m/s)
0,1,45.961101,-66.626891,683882.3328,5092463.889,-16.2781,4.5774,3.3034,8.4881,0.050,0.013889,0.010,0.002778,-0.035,0.009722
1,2,45.961102,-66.626891,683882.3138,5092463.955,-16.2921,4.5781,3.3032,8.4876,0.062,0.017222,0.012,0.003333,-0.035,0.009722
2,3,45.961103,-66.626892,683882.2510,5092464.085,-16.3845,4.5787,3.3031,8.4871,0.072,0.020000,0.011,0.003056,-0.035,0.009722
3,4,45.961104,-66.626891,683882.3262,5092464.165,-16.6144,4.5793,3.3029,8.4865,0.081,0.022500,0.008,0.002222,-0.035,0.009722
4,5,45.961106,-66.626890,683882.4332,5092464.348,-16.7938,4.5799,3.3027,8.4860,0.082,0.022778,0.003,0.000833,-0.035,0.009722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3441,3442,45.961085,-66.626886,683882.7745,5092462.080,-16.9812,3.8426,2.7180,5.0224,0.089,0.024722,-0.101,-0.028056,-0.163,0.045278
3442,3443,45.961084,-66.626884,683882.9252,5092461.994,-16.9722,3.8426,2.7180,5.0227,0.085,0.023611,-0.093,-0.025833,-0.165,0.045833
3443,3444,45.961083,-66.626883,683882.9931,5092461.850,-17.0140,3.8426,2.7180,5.0230,0.079,0.021944,-0.085,-0.023611,-0.164,0.045556
3444,3445,45.961083,-66.626884,683882.9536,5092461.846,-16.8004,3.8427,2.7180,5.0233,0.073,0.020278,-0.078,-0.021667,-0.163,0.045278


In [4]:
dt = 1

F = np.array([[1, dt], [0,1]])
G = np.array([[0.5], [1]])
HM = np.array([[1],[0]])

output = pd.DataFrame({"X":[]})

for row in df:
    Q = np.diag([df.iloc[row, 7]**2, 0.014**2])
    if row == 0:
        X_prediction = np.array([df.iloc[0, 3], df.iloc[row, 9]])
        P_prediction = np.diag([1,1])

    else:
        X_prediction = F * X_correction + G 
        P_prediction = F * P_correction * F.transpose() + Q
    
    Kalman_RHS = HM * P_prediction * HM.transpose() + df.iloc[row, 7]**2
    Kalman = P_prediction * HM.transpose() * Kalman_RHS

    Yx = HM * np.array([[df.iloc[row, 3]],[df.iloc[row, 9]]]) + df.iloc[row, 6]
    X_correction_RHS = Yx -HM * X_prediction
    X_correction = X_prediction + Kalman * X_correction_RHS
    output = output.append({"X": X_correction})

    P_correction_LHS = np.identity(2) - Kalman * HM
    P_correction = P_correction_LHS * P_prediction

output()
    
        



ValueError: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types

In [19]:
dt = 1

f = np.array([[1, dt], [0, 1]])
g = np.array([[0.5], [1]])
h = np.array([[1, 0]])

output = []

for idx, row in df.iterrows():
    q = np.diag([row['sdn(m)']**2, 0.014**2])
    if idx == 0:
        x_prediction = np.array([row['X (m)'], row['Vn (m/s)']])
        p_prediction = np.diag([1, 1])
    else:
        x_prediction = f @ x_correction + g 
        p_prediction = f @ p_correction @ f.T + q
    
    r = row['sdn(m)']**2
    kalman_rhs = h @ P_prediction @ h.T + r
    kalman = p_prediction @ h.T @ np.linalg.inv(kalman_rhs)

    Yx = h @ np.array([[row['X (m)']], [row['Vn (m/s)']]]) + row['sdn(m)']
    x_correction_rhs = Yx - h @ x_prediction
    x_correction = x_prediction + kalman @ x_correction_rhs
    output.append(x_correction)

    p_correction_lhs = np.identity(2) - kalman @ h
    p_correction = p_correction_lhs @ p_prediction

print(output)

[array([[6.83882541e+05, 2.22401864e-01],
       [6.83882333e+05, 1.38888890e-02]]), array([[654162.13322929, 713611.98409466],
       [652739.90667136,  31144.78945951]]), array([[715888.01616108, 687013.43853482],
       [626895.25331004,  28620.59408154]]), array([[651469.45941584, 682324.93571896],
       [597989.48924548,  27228.77948969]]), array([[718733.61188456, 685468.44571157],
       [576692.46521591,  26263.21773224]]), array([[650041.81421951, 682345.97877227],
       [551793.88648238,  25130.46983332]]), array([[720994.18089853, 685577.11437427],
       [533933.0568746 ,  24317.95709657]]), array([[648763.17537405, 682287.42113679],
       [512224.39114312,  23330.17656996]]), array([[722678.56344005, 685654.04729118],
       [497070.27528038,  22640.89380171]]), array([[647722.23585826, 682239.8066181 ],
       [477953.57163881,  21771.08908705]]), array([[723848.10774996, 685707.34115633],
       [464959.14787967,  21180.12230741]]), array([[646974.61835714, 682205.518

In [34]:
import pandas as pd
import numpy as np

dt = 1
f = np.array([[1, dt], [0, 1]])
g = np.array([[0.5], [1]])
h = np.array([[1, 0]])

output = pd.DataFrame(columns=['X', 'Y', 'H', 'Vx', 'Vy', 'Vh'])

for idx, row in df.iterrows():
    qx = np.diag([float(row['sdn(m)'])**2, 0.014**2])
    qy = np.diag([float(row['sde(m)'])**2, 0.014**2])
    qh = np.diag([float(row['sdu(m)'])**2, 0.014**2])

    if idx == 0:
        x_prediction = np.array([float(row['X (m)']), float(row['Vn (m/s)'])])
        px_prediction = np.diag([1, 1])
        y_prediction = np.array([float(row['Y (m)']), float(row['Ve(m/s)'])])
        py_prediction = np.diag([1, 1])
        h_prediction = np.array([float(row['height(m)']), float(row['Vu(m/s)'])])
        ph_prediction = np.diag([1, 1])

    else:
        x_prediction = f @ x_correction + g 
        px_prediction = f @ px_correction @ f.T + qx
        y_prediction = f @ y_correction + g 
        py_prediction = f @ py_correction @ f.T + qy
        h_prediction = f @ h_correction + g 
        ph_prediction = f @ ph_correction @ f.T + qh
    
    rx = float(row['sdn(m)'])**2
    ry = float(row['sde(m)'])**2
    rh = float(row['sdu(m)'])**2

    x_kalman_rhs = h @ p_prediction @ h.T + rx
    y_kalman_rhs = h @ p_prediction @ h.T + ry
    h_kalman_rhs = h @ p_prediction @ h.T + rh

    x_kalman = px_prediction @ h.T @ np.linalg.inv(x_kalman_rhs)
    y_kalman = py_prediction @ h.T @ np.linalg.inv(y_kalman_rhs)
    h_kalman = ph_prediction @ h.T @ np.linalg.inv(h_kalman_rhs)

    Yx = h @ np.array([[float(row['X (m)'])], [float(row['Vn (m/s)'])]]) + float(row['sdn(m)'])
    Yy = h @ np.array([[float(row['Y (m)'])], [float(row['Ve(m/s)'])]]) + float(row['sde(m)'])
    Yh = h @ np.array([[float(row['height(m)'])], [float(row['Vu(m/s)'])]]) + float(row['sdu(m)'])

    x_correction_rhs = Yx - h @ x_prediction
    x_correction_rhs1 = x_kalman @ x_correction_rhs
    x_correction = x_prediction + x_correction_rhs1

    y_correction_rhs = Yy - h @ y_prediction
    y_correction_rhs1 = y_kalman @ y_correction_rhs
    y_correction = y_prediction + y_correction_rhs1

    h_correction_rhs = Yh - h @ h_prediction
    h_correction_rhs1 = h_kalman @ h_correction_rhs
    h_correction = h_prediction + h_correction_rhs1

    output = output.append({"X": x_correction[0, 0], "Y": y_correction[0,0], "H": h_correction[0, 0], 
                            "Vx": x_correction[1,0], "Vy": y_correction[1,0], "Vh": h_correction[1, 0]}, ignore_index=True)

    px_correction_lhs = np.identity(2) - x_kalman @ h
    px_correction = px_correction_lhs @ px_prediction

    py_correction_lhs = np.identity(2) - y_kalman @ h
    py_correction = py_correction_lhs @ py_prediction

    ph_correction_lhs = np.identity(2) - h_kalman @ h
    ph_correction = ph_correction_lhs @ ph_prediction

print(output)



C:\Users\Avery\AppData\Local\Temp\ipykernel_18524\1668230943.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append({"X": x_correction[0, 0], "Y": y_correction[0,0], "H": h_correction[0, 0],
C:\Users\Avery\AppData\Local\Temp\ipykernel_18524\1668230943.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append({"X": x_correction[0, 0], "Y": y_correction[0,0], "H": h_correction[0, 0],
C:\Users\Avery\AppData\Local\Temp\ipykernel_18524\1668230943.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append({"X": x_correction[0, 0], "Y": y_correction[0,0], "H": h_correction[0, 0],
C:\Users\Avery\AppData\Local\Temp\ipykernel_18524\1668230943.py:60: FutureWarning: The frame.appe

                 X             Y           H             Vx            Vy  \
0     6.838824e+05  5.092464e+06  -16.189767  683882.332800  5.092464e+06   
1     1.019213e+06  8.308141e+06  -13.350860  668687.123742  4.946780e+06   
2     9.256733e+05  8.285777e+06   -9.062630  635934.478169  4.572445e+06   
3     9.603406e+05  8.106629e+06  -10.738471  610901.636823  4.233539e+06   
4     9.328138e+05  7.938969e+06   -9.563242  585314.099048  3.938266e+06   
...            ...           ...         ...            ...           ...   
3441  6.840749e+05  5.092670e+06  140.024921     305.768051  1.956204e+02   
3442  6.840749e+05  5.092670e+06  139.932900     305.568845  1.956154e+02   
3443  6.840748e+05  5.092670e+06  139.823205     305.370992  1.956102e+02   
3444  6.840747e+05  5.092670e+06  139.892163     305.174301  1.956054e+02   
3445  6.840746e+05  5.092670e+06  139.870653     304.979045  1.956010e+02   

              Vh  
0     -16.278100  
1     -15.026617  
2     -13.773043  

C:\Users\Avery\AppData\Local\Temp\ipykernel_18524\1668230943.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append({"X": x_correction[0, 0], "Y": y_correction[0,0], "H": h_correction[0, 0],
C:\Users\Avery\AppData\Local\Temp\ipykernel_18524\1668230943.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append({"X": x_correction[0, 0], "Y": y_correction[0,0], "H": h_correction[0, 0],
C:\Users\Avery\AppData\Local\Temp\ipykernel_18524\1668230943.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append({"X": x_correction[0, 0], "Y": y_correction[0,0], "H": h_correction[0, 0],
C:\Users\Avery\AppData\Local\Temp\ipykernel_18524\1668230943.py:60: FutureWarning: The frame.appe

In [33]:
output.to_csv('results2')
